# WATER DETECTION FROM OPTICAL IMAGERY

## MULTIBAND THRESHOLDING USING NDWI IS USED (REFER ATTACHED DOCUMENTATION FOR REASONS)

### SENTINEL-2 L2A CLOUD OPTIMIZED GEOTIFF IS USED FOR ANALYSIS
### IMAGERIES ARE LOADED USING STAC API
### RIOXARRAY AND GEOPANDAS ARE USED FOR IMAGE PROCESSING AND SPATIAL ANALYSIS

## TO RUN
### Run each cell one by one in the order.... That's it!

In [2]:
#importing libraries

from pystac_client import Client
from shapely.geometry import Point, Polygon
import rioxarray
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import box
import math
from shapely.ops import unary_union
import numpy as np
import time

In [3]:
#loading the csv data

df = pd.read_csv('ponds.csv')
df.columns = df.columns.str.strip()
df = df.rename(columns={'polygon': 'geometry'})
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326") #creating GeoDataFrame from pandas dataframe
gdf.head()

,Pond,Link,centriod,Unnamed: 3,geometry
0,1,https://www.google.com/maps?q=15.9097716016521...,"(15.909771601652196, 80.5101859752401)",NaN,"POLYGON ((80.51058 15.90978, 80.51058 15.90978..."
1,2,https://www.google.com/maps?q=15.9073785273048...,"(15.907378527304822, 80.51165072296921)",NaN,"POLYGON ((80.51101 15.90717, 80.51101 15.90738..."
2,3,https://www.google.com/maps?q=15.9080633321265...,"(15.908063332126533, 80.5111462520874)",NaN,"POLYGON ((80.51095 15.90804, 80.51095 15.90805..."
3,4,https://www.google.com/maps?q=15.9052441404584...,"(15.905244140458427, 80.6585489495667)",NaN,"POLYGON ((80.65834 15.90540, 80.65844 15.90538..."
4,5,https://www.google.com/maps?q=15.9003483917881...,"(15.900348391788192, 80.64620910177594)",NaN,"POLYGON ((80.64608 15.90051, 80.64608 15.90051..."


In [4]:
#To store constants, which we have to use at multiple places

class Constants:
    def __init__(self):
        self.AreaPerPixel=None
        self.crs=None
    
    def getAreaPerPixel(self):
        return self.AreaPerPixel
    
    def setAreaPerPixel(self, areaperpixel):
        self.AreaPerPixel=areaperpixel
        
    def getImageCRS(self):
        return self.crs
    
    def setImageCRS(self,crs):
        self.crs= crs

In [5]:
constants= Constants()  #creating object of constants

In [6]:
#Images are queried from Sentine-2 AWS repo using STAC API

api_url = "https://earth-search.aws.element84.com/v1"

def getImage(polygon_bounds, polygon_crs):
    returned=False
    try:
        client = Client.open(api_url)
        collection = "sentinel-2-l2a"
        search = client.search(
        collections=[collection],
        intersects=polygon_bounds,  #Acquisition tiles which intersects with bbox of polygon
        max_items=10,
        query={
            "eo:cloud_cover": {
            "lt": 20
                }
            }
        )
        items = search.item_collection()
        #If the polygon bounds are completely inside single acqusition, we can just return that of a particular date
        #If not, then we have to return multiple tiles and merge it
        #To check that, checking polygons are inside the bounds of tiles
        polygon_bbox= gpd.GeoDataFrame(geometry=[polygon_bounds],crs=polygon_crs)
        for i in range(0,len(items)):
            item_bounds= box(*items[i].bbox)
            item_bbox= gpd.GeoDataFrame(geometry=[item_bounds],crs=polygon_crs)
            if ((item_bbox.contains(polygon_bbox).any()) and (returned==False)):
                returned=True
                return items[i]
            else:
                continue
                #In this exercise, every polygons are completely within single tile, So :)
            
            
    except:
        print('STAC API Error')
        
        

In [7]:
#We dont need complete image. So subsetting image based on polygon's bounding box and taking required bands

def getImageSubset(image,polygdf):
    assets= image.assets
    green_href= assets['green'].href
    nir_href= assets['nir'].href
    green = rioxarray.open_rasterio(green_href)
    nir= rioxarray.open_rasterio(nir_href)
    
    constants.setImageCRS(str(green.rio.crs)) #All images in stac collection will havew same crs
    
    #converting polygon to gdf and to crs of image
    polygdf_reproj= polygdf.to_crs(constants.getImageCRS()) #reprojecting polygon w.r.t image
    polygon_bbox= polygdf_reproj.bounds
    
    gps=subsetParams(green,polygon_bbox) #green band subset params
    nps=subsetParams(nir,polygon_bbox) #nir band subset params
    
    green_sub = green[0,gps['yEnd']:gps['yStart'],gps['xStart']:gps['xEnd']].squeeze()
    nir_sub = nir[0,nps['yEnd']:nps['yStart'],nps['xStart']:nps['xEnd']].squeeze()
    
    return {'green':green_sub,
            'nir':nir_sub,
            }

In [8]:
#keeping subset params seperately to ensure different bands with different spatial resolutions are managed correctly

def subsetParams(band,polygon_bbox):
    #now calculating subsetting pixels from polygon
    pixelSizeX= band.rio.transform()[0]
    pixelSizeY= band.rio.transform()[4]
    x= band.rio.transform()[2]
    y= band.rio.transform()[5]
    constants.setAreaPerPixel(abs(pixelSizeX*pixelSizeY))
    
    xStart = math.floor((polygon_bbox['minx'] - x) / pixelSizeX)
    xEnd = math.ceil((polygon_bbox['maxx'] - x) / pixelSizeX)
    yStart = math.floor((polygon_bbox['miny'] - y) / pixelSizeY)
    yEnd = math.ceil((polygon_bbox['maxy'] - y) / pixelSizeY)
    
    return {'xStart':xStart,
            'xEnd':xEnd,
            'yStart':yStart,
            'yEnd':yEnd,
           }


In [9]:
#calculating NDWI

def calculateNDWI(green,nir):
    ndwi= (green-nir)/(green+nir)
    return ndwi

In [10]:
#all the pixel above the threshold is considered as water 

def calculateWater(ndwi,threshold):
    water_pixels = ndwi >= threshold
    water_pixel_count = water_pixels.sum().item()
    return water_pixel_count

### Main function here is the runAll() which is given below. Ensure that you have run all the cell above before running it

In [12]:
#Main function

def runAll():
    areaList=[]
    waterOrNot=[]
    datalen=len(gdf['geometry'])
    for i in range(0, datalen): 
        print(f'Working with area {i+1} /{datalen}')  #Total 64 polygons were there in this exercise
        
        polygon= gdf['geometry'][i]
        polygon_bounds= box(*polygon.bounds)
        image= getImage(polygon_bounds, gdf.crs)
        polygdf= gpd.GeoDataFrame(geometry=[polygon], crs=gdf.crs)
        
        #Till this point, we have only loaded data lazily using rioxarray, saving memory, time and computation
        #From here, rioxarray loads actual data from STAC server and do calculations
        item= getImageSubset(image,polygdf) #Returns a list contains both green and nir band details
        green=item['green']
        nir=item['nir']
        
        #reprojecting and combining featurecollection for clipping
        polygdf_reproj= polygdf.to_crs(constants.getImageCRS())
        clip_geom = unary_union(polygdf_reproj.geometry)
        
        clipped_green= green.rio.clip([clip_geom], polygdf_reproj.crs,drop=False,invert=False)
        clipped_nir= nir.rio.clip([clip_geom], polygdf_reproj.crs,drop=False,invert=False)
        clipped_green = clipped_green.astype(np.float32) #converting data from uint16 to float32 to capture right values
        clipped_nir = clipped_nir.astype(np.float32)
        
        ndwi = calculateNDWI(clipped_green,clipped_nir)
        ndwi = ndwi.where(np.isfinite(ndwi), np.nan) 
        count=calculateWater(ndwi,0)  #Here we are giving threshold of 1 for NDWI
        areaList.append(count*constants.getAreaPerPixel())  #area is in m^2
        if (count==0):
            waterOrNot.append(0)
        else:
            waterOrNot.append(1)
        
    return { 
        'water_status':waterOrNot,
        'area':areaList
            }


startTime=time.time()
result= runAll()
endTime=time.time()
print(f'Time taken: {endTime-startTime} seconds')

Working with area 1 /64
Working with area 2 /64
Working with area 3 /64
Working with area 4 /64
Working with area 5 /64
Working with area 6 /64
Working with area 7 /64
Working with area 8 /64
Working with area 9 /64
Working with area 10 /64
Working with area 11 /64
Working with area 12 /64
Working with area 13 /64
Working with area 14 /64
Working with area 15 /64
Working with area 16 /64
Working with area 17 /64
Working with area 18 /64
Working with area 19 /64
Working with area 20 /64
Working with area 21 /64
Working with area 22 /64
Working with area 23 /64
Working with area 24 /64
Working with area 25 /64
Working with area 26 /64
Working with area 27 /64
Working with area 28 /64
Working with area 29 /64
Working with area 30 /64
Working with area 31 /64
Working with area 32 /64
Working with area 33 /64
Working with area 34 /64
Working with area 35 /64
Working with area 36 /64
Working with area 37 /64
Working with area 38 /64
Working with area 39 /64
Working with area 40 /64
Working w

In [13]:
#Add result to dataframe
df['water_status']=result['water_status']
df['area']=result['area']

In [14]:
df #see the data

,Pond,Link,centriod,Unnamed: 3,geometry,water_status,area
0,1,https://www.google.com/maps?q=15.9097716016521...,"(15.909771601652196, 80.5101859752401)",NaN,"POLYGON ((80.51058 15.90978, 80.51058 15.90978...",0,0.0
1,2,https://www.google.com/maps?q=15.9073785273048...,"(15.907378527304822, 80.51165072296921)",NaN,"POLYGON ((80.51101 15.90717, 80.51101 15.90738...",0,0.0
2,3,https://www.google.com/maps?q=15.9080633321265...,"(15.908063332126533, 80.5111462520874)",NaN,"POLYGON ((80.51095 15.90804, 80.51095 15.90805...",0,0.0
3,4,https://www.google.com/maps?q=15.9052441404584...,"(15.905244140458427, 80.6585489495667)",NaN,"POLYGON ((80.65834 15.90540, 80.65844 15.90538...",1,1000.0
4,5,https://www.google.com/maps?q=15.9003483917881...,"(15.900348391788192, 80.64620910177594)",NaN,"POLYGON ((80.64608 15.90051, 80.64608 15.90051...",1,2100.0
...,...,...,...,...,...,...,...
59,60,https://www.google.com/maps?q=16.1881587718507...,"(16.188158771850727, 81.11642739689296)",NaN,"POLYGON ((81.11643 16.18837, 81.11661 16.18846...",1,1000.0
60,61,https://www.google.com/maps?q=16.1920629513337...,"(16.1920629513337, 81.12248814387651)",NaN,"POLYGON ((81.12223 16.19243, 81.12251 16.19243...",1,3300.0
61,62,https://www.google.com/maps?q=16.1920629513337...,"(16.1920629513337, 81.12248814387651)",NaN,"POLYGON ((81.12223 16.19243, 81.12251 16.19243...",1,3300.0
62,63,https://www.google.com/maps?q=16.1927015132574...,"(16.192701513257482, 81.12152041321168)",NaN,"POLYGON ((81.12176 16.19288, 81.12176 16.19270...",1,1600.0


In [15]:
df.to_csv('water_detect_result.csv')  #save the data